In [1]:
size_system = 5

import pickle
import numpy as np
from stability_hs import *
import sympy as sp

with open('lf0_old_coord.pickle', 'rb') as pickle_file:
    P1_sym_old_coord = pickle.load(pickle_file)
with open('lf1_old_coord.pickle', 'rb') as pickle_file:
    P2_sym_old_coord = pickle.load(pickle_file)
with open('vector_sw_pr_mode0_less0_old_coord.pickle', 'rb') as pickle_file:
    vector_sw_pr_mode0_less0 = pickle.load(pickle_file)
with open('lambdas.pickle', 'rb') as pickle_file:
    lambdas = pickle.load(pickle_file)
with open('eta2.pickle', 'rb') as pickle_file:
    eta2 = pickle.load(pickle_file)
with open('mu2.pickle', 'rb') as pickle_file:
    mu2 = pickle.load(pickle_file)

from scipy import io
HybridSystemMatlab = io.loadmat(f"PI_controller_converter/Reformulate/data_matlab/data_to_python_size_{size_system}.mat")

def reformulate_PI(A_or, B_or, C_or, Invar_or, KP_or, KI_or, num_var, num_con, num_out, ref_val):
    # We reformulate a PI controller of the form x' = A_or * x + B_or * u 
    # (_or stands for original)
    # in the new variable w = [x; u], obtaining the dynsys of the form
    #  w' = A_homo * w + b_homo.
    # Also the linear condition of the switch is modified accordingly (does not
    # care about the value of u).

    A_homo_top = np.hstack([A_or, B_or])
    N = -1 * np.dot(KP_or, np.dot(C_or, A_or)) - np.dot(KI_or, C_or)
    M = -1 * np.dot(KP_or, np.dot(C_or, B_or))
    A_homo_bot = np.hstack([N, M])
    A_homo = np.vstack([A_homo_top, A_homo_bot])

    b_bot = np.dot(KI_or, ref_val)
    b_homo = np.vstack([np.zeros([num_var,1]), b_bot])

    C_homo = np.hstack([C_or, np.zeros([num_out, num_con])])

    Invar_geq0_homo = np.hstack([Invar_or[:,:num_var], np.zeros([len(Invar_or), num_con]), Invar_or[:,[-1]] ])

    return (A_homo, b_homo, C_homo, Invar_geq0_homo)

In [2]:

# We format the loaded data in python files
num_modes = HybridSystemMatlab["num_modes"][0][0]
num_variables = HybridSystemMatlab["num_variables"][0][0]
num_controllers = HybridSystemMatlab["num_controllers"][0][0]
num_outputs = HybridSystemMatlab["num_outputs"][0][0]
reference_values = HybridSystemMatlab["reference_values"]
# reference_values = np.asarray(np.matrix([[0.5],[0],[-1],[20]]))
As = []
Bs = []
Cs = []
KIs = []
KPs = []
Invars_geq0 = []
for ind_mode in range(num_modes):
    As.append(HybridSystemMatlab[f"A_{ind_mode}"])
    Bs.append(HybridSystemMatlab[f"B_{ind_mode}"])
    Cs.append(HybridSystemMatlab[f"C_{ind_mode}"])
    KIs.append(HybridSystemMatlab[f"KI_{ind_mode}"])
    KPs.append(HybridSystemMatlab[f"KP_{ind_mode}"])
    Invars_geq0.append(HybridSystemMatlab[f"Invar_{ind_mode}_geq0"])
if size_system == 18:
    np.set_printoptions(precision=3, linewidth=1000, suppress=True)
else:
    np.set_printoptions(precision=3, linewidth=1000, suppress=True)

logging.info(f"\n\n\nThe system has \n{num_modes} modes, \n{num_variables} variables, \n{num_controllers} controllers, \n{num_outputs} outputs.\nReference values are {np.transpose(reference_values)}.\n")

logging.info(f"Mode 0: g * [x;1] >= 0, where g = {Invars_geq0[0]}")
logging.info(f"The dynamics is x_dot = A x + B u, where\n A = \n{As[0]}\n and \nB = \n{Bs[0]}.\nThe control is given by \n K_P = \n{KPs[0]}\n and \nK_I = \n{KIs[0]}.\n.")
logging.info(f"Mode 1: g * [x;1] >= 0, where g = {Invars_geq0[1]}")
logging.info(f"The dynamics is x_dot = A x + B u, where\n A = \n{As[1]}\n and \nB = \n{Bs[1]}.\nThe control is given by \n K_P = \n{KPs[1]}\n and \nK_I = \n{KIs[1]}.\n.")

num_homo_variables = num_variables + num_controllers

# We reformulate the system with PI controller in a homogenous system
As_homo = []
bs_homo = []
Cs_homo = []
Invars_geq0_homo = []

for ind_mode in range(num_modes):
    (A_homo, b_homo, C_homo, Invar_geq0_homo) = reformulate_PI(As[ind_mode], Bs[ind_mode], Cs[ind_mode], 
                                                            Invars_geq0[ind_mode], KPs[ind_mode], KIs[ind_mode], 
                                                            num_variables, num_controllers, num_outputs, reference_values)

    As_homo.append(A_homo)
    bs_homo.append(b_homo)
    Cs_homo.append(C_homo)
    Invars_geq0_homo.append(Invar_geq0_homo)


In [3]:
b_bot = np.dot(KIs[ind_mode], reference_values)
b_homo = np.vstack([np.zeros([num_variables,1]), b_bot])

In [4]:
b_homo

array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [ 100.],
       [-100.],
       [  40.]])

In [14]:
old_refs = pc.Constant("old_refs", reference_values.astype(np.double), shape=(4,1))

sdp = pc.Problem()

change_in_r = 0 // pc.RealVariable("change_in_r", shape = (3, 1))

sdp.maximize = sum(change_in_r)

new_refs = old_refs + change_in_r

A0_const = pc.Constant("A0_const", As[0].astype(np.double))
B0_const = pc.Constant("B0_const", Bs[0].astype(np.double))

A0_homo_top_const = A0_const & B0_const

N0 = -1 * np.dot(KPs[0], np.dot(Cs[0], As[0])) - np.dot(KIs[0], Cs[0])
M0 = -1 * np.dot(KPs[0], np.dot(Cs[0], Bs[0]))

N0_const = pc.Constant("N0_const", N0.astype(np.double))
M0_const = pc.Constant("M0_const", M0.astype(np.double))

A0_homo_bot = N0_const & M0_const
A0_homo = A0_homo_top_const // A0_homo_bot

b0_bot = pc.Constant("KIs0", np.matrix(KIs[0], float)) * new_refs
b0_homo = pc.Constant("zeros", np.zeros([n-4,1])) // b0_bot

A0bar_old = (A0_homo & b0_homo) // (0 & 0)


A1_const = pc.Constant("A1_const", As[1].astype(np.double))
B1_const = pc.Constant("B1_const", Bs[1].astype(np.double))

A1_homo_top_const = A1_const & B1_const

N1 = -1 * np.dot(KPs[1], np.dot(Cs[1], As[1])) - np.dot(KIs[1], Cs[1])
M1 = -1 * np.dot(KPs[1], np.dot(Cs[1], Bs[1]))

N1_const = pc.Constant("N1_const", N1.astype(np.double))
M1_const = pc.Constant("M1_const", M1.astype(np.double))

A1_homo_bot = N1_const & M1_const
A1_homo = A1_homo_top_const // A1_homo_bot

b1_bot = pc.Constant("KIs1", np.matrix(KIs[1], float)) * new_refs
b1_homo = pc.Constant("zeros", np.zeros([n-4,1])) // b1_bot

A1bar_old = (A1_homo & b1_homo) // (0 & 0)

vector_sw_pr_mode0_geq0 = -vector_sw_pr_mode0_less0

In [25]:
n = size_system + 4
theta = 1



P1 = pc.Constant("P1_old_coord", np.matrix(P1_sym_old_coord,float))

A0_homo_inv = pc.Constant("A_0_homo_inv", np.linalg.inv(A0_homo.np))
Bnow = pc.Constant("zeros", np.zeros([n-4,4])) // pc.Constant("KIs0", np.matrix(KIs[0], float))
translation_sdp = (pc.Constant("id", np.eye(n-1)) & - A0_homo_inv * Bnow * change_in_r) // (0 & 1)

P1 = translation_sdp.T * P1 * translation_sdp

C0 = pc.Constant("C0", np.transpose(C_homo[0]))

cd = (C0 // new_refs[0]).T

can_vects = []
for k in range(n):
    can_vects.append( pc.Constant(f"canvec{k}", np.matrix(can_vec_sp(k+1, n),float)))

# cd = new_refs[0]
# for k in range(n-2, -1, -1):
#     cd = C_homo[0][k] & cd
Qcd = []
for k in range(n):
    Qcd.append( can_vects[k] * cd + (can_vects[k] * cd).T)

lambdas_const = pc.Constant("lambdas_const", lambdas)

P2b = P1
for index in range(n):
    P2b = P2b + lambdas_const[index] * Qcd[index]

Zeros = pc.Constant("Zeros", np.zeros([n-1,n-1]))
s = np.matrix(sp.Matrix(vector_sw_pr_mode0_geq0[:-1])/2)
vec = pc.Constant("vector_sw_pr_mode0_geq0_halved",  s.astype(np.double))

Q1 = (Zeros & vec) // (vec.T &  theta - new_refs[0] )

Q2 = (Zeros & -vec) // (-vec.T &  -theta + new_refs[0] )

eta2_const = pc.Constant("eta2", eta2)
mu2_const = pc.Constant("mu2", mu2)

Cond2 = P2b - mu2 * Q2

sdp.add_constraint(Cond2 >> 0)

Cond6 = A1bar_old.T * P2b + P2b * A1bar_old + eta2 * Q2

# sdp.add_constraint(Cond6 << 0)

# QQ1A = sp.Matrix.vstack(
#         sp.Matrix.hstack(sp.Matrix.zeros(n-1,n-1), (sp.Matrix(vector_sw_pr_mode0_geq0[:-1])/2)),
#         sp.Matrix.hstack(sp.Matrix(vector_sw_pr_mode0_geq0[:-1]).transpose()/2, 
#                          sp.Matrix([theta - new_refs[0]]))
#     )
# QQ2A = sp.Matrix.vstack(
#     sp.Matrix.hstack(sp.Matrix.zeros(n-1,n-1), -(sp.Matrix(vector_sw_pr_mode0_geq0[:-1])/2)),
#     sp.Matrix.hstack(-sp.Matrix(vector_sw_pr_mode0_geq0[:-1]).transpose()/2, 
#                         -sp.Matrix([vector_sw_pr_mode0_geq0[-1]]))
#     )





NotImplementedError: PICOS does not support multidimensional quadratic expressions at this point. More precisely, one factor must be constant or the result must be scalar.

In [23]:
P2b - mu2 * Q2

<9×9 Real Constant: P1_old_coord + lambdas_const[0]·(canvec0·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec0·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[1]·(canvec1·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec1·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[2]·(canvec2·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec2·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[3]·(canvec3·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec3·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[4]·(canvec4·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec4·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[5]·(canvec5·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec5·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[6]·(canvec6·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec6·[C0; (old_refs + [0; change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[7]·(canvec7·[C0; (old_refs + [0; change_in_r])[0]]ᵀ + (canvec7·[C0; (old_refs + [0; change_in_r])

In [21]:
print(sdp)

Semidefinite Program
  maximize [0; change_in_r][0] + [0; change_in_r][1] + [0; change_in_r][2] + [0; change_in_r][3]
  over
    3×1 real variable change_in_r
  subject to
    P1_old_coord + lambdas_const[0]·(canvec0·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ + (canvec0·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[1]·(canvec1·[C0; (old_refs
      + [0; change_in_r])[0]]ᵀ + (canvec1·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[2]·(canvec2·[C0; (old_refs
      + [0; change_in_r])[0]]ᵀ + (canvec2·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[3]·(canvec3·[C0; (old_refs
      + [0; change_in_r])[0]]ᵀ + (canvec3·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[4]·(canvec4·[C0; (old_refs
      + [0; change_in_r])[0]]ᵀ + (canvec4·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ)ᵀ) + lambdas_const[5]·(canvec5·[C0; (old_refs
      + [0; change_in_r])[0]]ᵀ + (canvec5·[C0; (old_refs + [0;
      change_in_r])[0]]ᵀ)ᵀ) + lambd

In [22]:
solution = sdp.solve(solver = 'cvxopt')

SolutionFailure: Code 3: Primal solution state claimed empty but optimality is required (primals=True).

In [380]:
new_refs.np

array([ 0.   ,  5.203, -0.987, 19.988])

In [9]:
lambdas = pc.RealVariable("lambdas", n)

P2b = pc.SymmetricVariable("P2b", n)

cd = sp.Matrix.hstack(QQ1[-1,:-1]*2, sp.Matrix([QQ1[-1,-1]]))
Qcd_consts = {}
for index in range(n):
    Qcd_consts[index] = pc.Constant(f"Qcd{index}", np.matrix(Qcd[index], float))

expression_P2b = P1b
for index in range(n):
    expression_P2b = expression_P2b + lambdas[index] * Qcd_consts[index]

sdp.add_constraint(P2b == expression_P2b)

A1b = pc.Constant("A1b", np.matrix(A1bar, float))
A2b = pc.Constant("A2b", np.matrix(A2bar, float))

# alpha = pc.RealVariable("alpha")
# beta = pc.RealVariable("beta")
# gamma = pc.RealVariable("gamma")
# sdp.add_constraint(alpha >> 0)
# sdp.add_constraint(beta >> 0)
# sdp.add_constraint(gamma >> 0)

# Q1 = pc.Constant("Q1", QQ1)
Q2 = pc.Constant("Q2", np.matrix(QQ2, float))

# I = pc.Constant("I", Ibar)

# mu1 = pc.RealVariable("mu1")
# sdp.add_constraint(mu1 >> 0)
mu2 = pc.RealVariable("mu2")
sdp.add_constraint(mu2 >> 0)

# eta1 = pc.RealVariable("eta1")
# sdp.add_constraint(eta1 >> 0)
eta2 = pc.RealVariable("eta2")
sdp.add_constraint(eta2 >> 0)

if normalize == True:
    # The following is a normalization condition. It needs to hold (easy to see with Sylvester method).
    # Sometimes the solver does not provide a solution if we give it this condition, but does provide
    # a solution if we do not. This is strange and could be investigated.
    sdp.add_constraint((P1b[0,0]) >> 1)

Cond1 = P1b 
Cond2 = P2b - mu2 * Q2  

Cond5 = A1b.T * P1b + P1b * A1b 
Cond6 = A2b.T * P2b + P2b * A2b + eta2 * Q2 



# conditions 3.5
sdp.add_constraint(Cond1 >> 0)
sdp.add_constraint(Cond2 >> 0)


# conditions 3.7
sdp.add_constraint(Cond5 << 0)
sdp.add_constraint(Cond6 << 0)

sol = sdp.solve(solver=sdp_solver)


array([[ 0.5],
       [ 5. ],
       [-1. ],
       [20. ]])